# DeepLabV3+

In [ ]:
!pip install torch torchvision torchvision albumentations matplotlib opencv-python numpy

### Define the dataset class

In [ ]:
import torch
import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Define transformation
transform = A.Compose([
    A.Resize(512, 512), # resizing image to 512x512 pixels
    A.Normalize(mean=(0.5,), std=(0.5,)), # normalizing pixel values ([0, 255]->[-1, 1])
    ToTensorV2() # converts images to pytorch tensors
])

class DropletDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir # path to the folder containing input images
        self.mask_dir = mask_dir
        self.transform = transform # to apply tranformations
        self.images = os.listdir(image_dir) # Lists all filenames in image_dir (assuming masks have the same names)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # given an item index, retrieves its image path and mask path
        img_name = self.images[idx]  # Get image filename
        mask_name = img_name.rsplit(".", 1)[0] + "_mask." + img_name.rsplit(".", 1)[1]  # Replace extension correctly
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, mask_name)
        
        image = cv2.cvtColor(cv2.imread(img_path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB) # reads image in RGB
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) # reads mask in greyscale

        if image is None or mask is None:
            raise ValueError(f"Failed to load image or mask: {img_path}, {mask_path}")

        if self.transform:
            # apply transformations to both image and mask
            augmented = self.transform(image=image, mask=mask)
            image, mask = augmented["image"], augmented["mask"]

        return image, mask.long()


### Load the dataset

In [ ]:
train_loader = DataLoader(DropletDataset("../../dataset_120_real", "../segmentation_masks", transform), batch_size=4, shuffle=True)

### Load the pretrained model

In [ ]:
from torchvision import models

model = models.segmentation.deeplabv3_resnet50(pretrained=True)
model.classifier[4] = torch.nn.Conv2d(256, 1, kernel_size=(1,1))  # Adjust output for binary segmentation
#model = model.cuda()

### Train the model

Define the loss function

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

def dice_loss(pred, target):
    smooth = 1.0
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth))

criterion = lambda pred, target: 0.5 * F.binary_cross_entropy_with_logits(pred, target.float()) + dice_loss(pred, target)
optimizer = optim.Adam(model.parameters(), lr=1e-4)


Actually train the model

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        # images, masks = images.cuda(), masks.cuda()
        
        optimizer.zero_grad()
        outputs = model(images)["out"]
        loss = criterion(outputs.squeeze(1), masks)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

[ WARN:0@447.912] global loadsave.cpp:268 findDecoder imread_('../segmentation_masks/frame_106.png'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'astype'

### Evaluate the model